# Day 1, Session 4: LLM-Powered Decision Making

## From Static Workflows to Intelligent Routing

Traditional workflows follow fixed paths. But real invoice processing requires intelligent decisions:
- Should this invoice route to OCR or direct text extraction?
- Which validation rules apply to this specific vendor?
- Does this document need human review or can it auto-approve?

**Today we build graphs that think.**

### What We're Building

An intelligent invoice router that:
1. **Analyzes** document characteristics
2. **Decides** optimal processing path
3. **Adapts** validation rules dynamically
4. **Routes** to appropriate approval workflows
5. **Learns** from processing outcomes

This is how production AI systems make decisions at scale.

**Duration: 15 minutes**

In [ ]:
# Global configuration - Instructor will fill these
OLLAMA_URL = "http://XX.XX.XX.XX"  # Course server IP (port 80)
API_TOKEN = "YOUR_TOKEN_HERE"      # Instructor provides token
MODEL = "qwen3:8b"                  # Default model on server

In [ ]:
# Install required packages
!pip install -q langgraph langchain-core
!pip install -q pydantic typing-extensions

In [ ]:
import requests
import json
import time
from typing import Dict, List, Optional, Any, TypedDict, Literal
from pydantic import BaseModel, Field
from enum import Enum
from datetime import datetime

from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

# Health check
def check_server_health():
    """Verify server connection"""
    try:
        response = requests.get(f"{OLLAMA_URL}/health")
        if response.status_code == 200:
            data = response.json()
            print(f"✅ Server Status: {data.get('status', 'Unknown')}")
            print(f"📊 Models Available: {data.get('models_count', 0)}")
            return True
    except Exception as e:
        print(f"❌ Server connection failed: {e}")
    return False

# LLM calling function
def call_llm(prompt: str, model: str = MODEL) -> str:
    """Call the LLM with a prompt"""
    headers = {
        "Authorization": f"Bearer {API_TOKEN}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": model,
        "prompt": prompt
    }
    
    try:
        response = requests.post(
            f"{OLLAMA_URL}/think",
            headers=headers,
            json=data
        )
        if response.status_code == 200:
            return response.json().get('response', '')
        else:
            return f"Error: {response.status_code}"
    except Exception as e:
        return f"Error: {e}"

print("🤖 LLM-Powered Decision Making Demo")
print("🔌 Connecting to course server...")
server_available = check_server_health()

if server_available:
    print("\n🧠 Testing LLM connection...")
    test_response = call_llm("Hello! Respond with: 'Intelligent routing system ready.'")
    print(f"Response: {test_response[:100]}...")
else:
    print("\n⚠️ Will use mock responses for demo")

## Step 1: Document Analysis for Intelligent Routing

First, we need to understand document characteristics to make routing decisions.

In [ ]:
class DocumentComplexity(Enum):
    SIMPLE = "simple"          # Standard invoice format
    MODERATE = "moderate"      # Multiple line items, some formatting
    COMPLEX = "complex"        # Non-standard layout, multiple pages
    CHALLENGING = "challenging" # Handwritten, poor quality, foreign language

class ProcessingPath(Enum):
    FAST_TRACK = "fast_track"       # Simple extraction + basic validation
    STANDARD = "standard"           # Full OCR + AI extraction + validation  
    ENHANCED = "enhanced"           # Multiple models + human-in-loop
    MANUAL = "manual"               # Human processing required

class DocumentAnalysis(BaseModel):
    """Structured analysis of document characteristics"""
    complexity: DocumentComplexity
    confidence: float = Field(ge=0.0, le=1.0)
    text_quality: str  # 'high', 'medium', 'low'
    layout_standard: bool
    language: str
    page_count: int
    vendor_known: bool
    processing_path: ProcessingPath
    reasoning: str

def analyze_document_with_llm(document_info: Dict[str, Any]) -> DocumentAnalysis:
    """Use LLM to analyze document and determine processing strategy"""
    
    analysis_prompt = f"""
You are an intelligent document routing system. Analyze this invoice document and determine the optimal processing path.

Document Information:
- Text Content: {document_info.get('text', 'No text provided')[:500]}...
- Image Quality: {document_info.get('image_quality', 'unknown')}
- Page Count: {document_info.get('page_count', 1)}
- File Type: {document_info.get('file_type', 'unknown')}
- File Size: {document_info.get('file_size', 'unknown')}

Analyze and classify:

1. COMPLEXITY ASSESSMENT:
   - simple: Standard invoice format, clear text, single page
   - moderate: Multiple line items, good formatting, 1-3 pages
   - complex: Non-standard layout, multiple pages, some formatting issues
   - challenging: Poor quality, handwritten elements, unusual format

2. PROCESSING PATH RECOMMENDATION:
   - fast_track: Simple extraction, basic validation (for simple docs)
   - standard: Full OCR + AI extraction + validation (for moderate docs)
   - enhanced: Multiple models + validation + review (for complex docs)
   - manual: Human processing required (for challenging docs)

3. KEY FACTORS:
   - Text quality assessment
   - Layout standardization
   - Language detection
   - Vendor recognition
   
Respond in JSON format:
{{
    "complexity": "simple|moderate|complex|challenging",
    "confidence": 0.95,
    "text_quality": "high|medium|low",
    "layout_standard": true,
    "language": "english",
    "page_count": 1,
    "vendor_known": false,
    "processing_path": "fast_track|standard|enhanced|manual",
    "reasoning": "Brief explanation of decision"
}}
"""

    if server_available:
        response = call_llm(analysis_prompt)
        try:
            # Extract JSON from response
            json_start = response.find('{')
            json_end = response.rfind('}') + 1
            if json_start != -1 and json_end > json_start:
                json_str = response[json_start:json_end]
                analysis_data = json.loads(json_str)
                return DocumentAnalysis(**analysis_data)
        except (json.JSONDecodeError, ValueError) as e:
            print(f"Error parsing LLM response: {e}")
    
    # Fallback analysis
    text_length = len(document_info.get('text', ''))
    if text_length < 100:
        complexity = DocumentComplexity.CHALLENGING
        path = ProcessingPath.MANUAL
    elif text_length < 500:
        complexity = DocumentComplexity.SIMPLE
        path = ProcessingPath.FAST_TRACK
    elif text_length < 2000:
        complexity = DocumentComplexity.MODERATE
        path = ProcessingPath.STANDARD
    else:
        complexity = DocumentComplexity.COMPLEX
        path = ProcessingPath.ENHANCED
    
    return DocumentAnalysis(
        complexity=complexity,
        confidence=0.8,
        text_quality="medium",
        layout_standard=True,
        language="english",
        page_count=document_info.get('page_count', 1),
        vendor_known=False,
        processing_path=path,
        reasoning="Fallback analysis based on text length"
    )

# Test document analysis
print("🔍 Testing Intelligent Document Analysis")
print("=" * 50)

# Sample invoice documents with different characteristics
test_documents = [
    {
        "name": "Simple Invoice",
        "text": "INVOICE #001 Date: 2024-01-15 From: TechCorp To: ABC Inc Amount: $1,500 Due: 2024-02-15",
        "image_quality": "high",
        "page_count": 1,
        "file_type": "PDF"
    },
    {
        "name": "Complex Multi-page Invoice", 
        "text": "FACTURA COMERCIAL No. 2024-0156 Fecha: 15 enero 2024 Empresa: Suministros... [extensive Spanish text with complex line items, multiple tax calculations, delivery terms, payment schedules across 3 pages with various formatting inconsistencies]",
        "image_quality": "medium",
        "page_count": 3,
        "file_type": "scanned PDF"
    },
    {
        "name": "Poor Quality Receipt",
        "text": "Rec... $12.5 handwritten notes unclear scan",
        "image_quality": "low", 
        "page_count": 1,
        "file_type": "photo"
    }
]

for doc in test_documents:
    print(f"\n📄 Analyzing: {doc['name']}")
    analysis = analyze_document_with_llm(doc)
    
    print(f"   Complexity: {analysis.complexity.value}")
    print(f"   Processing Path: {analysis.processing_path.value}")
    print(f"   Confidence: {analysis.confidence:.2f}")
    print(f"   Reasoning: {analysis.reasoning}")

## Step 2: Dynamic Validation Rule Selection

Different vendors and document types require different validation approaches.

In [ ]:
class ValidationRuleSet(BaseModel):
    """Set of validation rules to apply"""
    vendor_verification: bool
    amount_thresholds: Dict[str, float]
    required_fields: List[str]
    tax_validation: bool
    date_logic_checks: bool
    duplicate_detection: bool
    po_matching_required: bool
    custom_rules: List[str]
    confidence_threshold: float

def select_validation_rules_with_llm(document_analysis: DocumentAnalysis, vendor_info: Dict) -> ValidationRuleSet:
    """Use LLM to dynamically select appropriate validation rules"""
    
    selection_prompt = f"""
You are an intelligent validation rule selector for invoice processing. Based on the document analysis and vendor information, determine which validation rules should be applied.

Document Analysis:
- Complexity: {document_analysis.complexity.value}
- Processing Path: {document_analysis.processing_path.value}
- Vendor Known: {document_analysis.vendor_known}
- Text Quality: {document_analysis.text_quality}
- Confidence: {document_analysis.confidence}

Vendor Information:
- Name: {vendor_info.get('name', 'Unknown')}
- Trust Level: {vendor_info.get('trust_level', 'unknown')}
- Contract Amount: {vendor_info.get('contract_amount', 0)}
- Previous Issues: {vendor_info.get('previous_issues', 0)}
- Payment History: {vendor_info.get('payment_history', 'unknown')}

VALIDATION RULE SELECTION CRITERIA:

1. Vendor Verification:
   - Required for unknown/low-trust vendors
   - Optional for trusted long-term partners

2. Amount Thresholds:
   - Trusted vendors: Higher auto-approval limits
   - New vendors: Lower limits, more scrutiny
   - High-risk vendors: Manual review for any amount

3. Required Fields:
   - Simple docs: Basic fields (invoice#, amount, vendor)
   - Complex docs: Extended fields (PO#, line items, tax details)
   - Regulatory compliance: Additional fields as needed

4. Tax Validation:
   - Required for amounts > $1000
   - Optional for trusted small vendors
   - Enhanced for international vendors

5. Duplicate Detection:
   - Always enabled for amounts > $500
   - Enhanced for vendors with previous duplicate issues

6. PO Matching:
   - Required for contract vendors
   - Optional for service vendors
   - Not required for utilities/subscriptions

Respond in JSON format:
{{
    "vendor_verification": true,
    "amount_thresholds": {{
        "auto_approve": 1000.0,
        "manager_approval": 10000.0,
        "director_approval": 50000.0
    }},
    "required_fields": ["invoice_number", "amount", "vendor", "date"],
    "tax_validation": true,
    "date_logic_checks": true,
    "duplicate_detection": true,
    "po_matching_required": false,
    "custom_rules": ["check_contract_compliance"],
    "confidence_threshold": 0.85
}}
"""

    if server_available:
        response = call_llm(selection_prompt)
        try:
            json_start = response.find('{')
            json_end = response.rfind('}') + 1
            if json_start != -1 and json_end > json_start:
                json_str = response[json_start:json_end]
                rules_data = json.loads(json_str)
                return ValidationRuleSet(**rules_data)
        except (json.JSONDecodeError, ValueError) as e:
            print(f"Error parsing LLM response: {e}")

    # Fallback rule selection based on complexity
    if document_analysis.complexity == DocumentComplexity.SIMPLE:
        return ValidationRuleSet(
            vendor_verification=False,
            amount_thresholds={"auto_approve": 5000.0, "manager_approval": 25000.0},
            required_fields=["invoice_number", "amount", "vendor"],
            tax_validation=False,
            date_logic_checks=True,
            duplicate_detection=True,
            po_matching_required=False,
            custom_rules=[],
            confidence_threshold=0.8
        )
    else:
        return ValidationRuleSet(
            vendor_verification=True,
            amount_thresholds={"auto_approve": 1000.0, "manager_approval": 10000.0, "director_approval": 50000.0},
            required_fields=["invoice_number", "amount", "vendor", "date", "line_items"],
            tax_validation=True,
            date_logic_checks=True,
            duplicate_detection=True,
            po_matching_required=True,
            custom_rules=["enhanced_verification"],
            confidence_threshold=0.9
        )

# Test validation rule selection
print("\n⚖️ Testing Dynamic Validation Rule Selection")
print("=" * 50)

# Sample vendor scenarios
vendor_scenarios = [
    {
        "name": "TrustedVendor Corp",
        "trust_level": "high", 
        "contract_amount": 100000,
        "previous_issues": 0,
        "payment_history": "excellent"
    },
    {
        "name": "NewSupplier LLC",
        "trust_level": "unknown",
        "contract_amount": 0,
        "previous_issues": 0,
        "payment_history": "none"
    },
    {
        "name": "ProblematicVendor Inc",
        "trust_level": "low",
        "contract_amount": 50000,
        "previous_issues": 3,
        "payment_history": "poor"
    }
]

# Test with different document complexities and vendor types
for i, vendor in enumerate(vendor_scenarios):
    doc_analysis = analyze_document_with_llm(test_documents[i % len(test_documents)])
    rules = select_validation_rules_with_llm(doc_analysis, vendor)
    
    print(f"\n🏢 Vendor: {vendor['name']} (Trust: {vendor['trust_level']})")
    print(f"   Document: {doc_analysis.complexity.value}")
    print(f"   Auto-approve limit: ${rules.amount_thresholds.get('auto_approve', 0):,.0f}")
    print(f"   Required fields: {len(rules.required_fields)}")
    print(f"   PO matching: {'Yes' if rules.po_matching_required else 'No'}")
    print(f"   Custom rules: {rules.custom_rules}")

## Step 3: Conditional Routing Logic

Build a graph that routes intelligently based on analysis and context.

In [ ]:
# Define the intelligent processing state
class IntelligentProcessingState(TypedDict):
    # Input
    document_info: Dict[str, Any]
    vendor_context: Dict[str, Any]
    
    # Analysis results
    document_analysis: Optional[DocumentAnalysis]
    validation_rules: Optional[ValidationRuleSet]
    
    # Processing decisions
    processing_path: Optional[str]
    approval_route: Optional[str]
    
    # Results
    extracted_data: Optional[Dict]
    validation_results: Optional[Dict]
    final_decision: Optional[str]
    
    # Audit trail
    decision_log: List[Dict]
    processing_time: float

class IntelligentRouter:
    """LLM-powered intelligent routing system"""
    
    def __init__(self):
        self.graph = self._build_intelligent_graph()
        self.processing_stats = {"decisions_made": 0, "routing_changes": 0}
    
    def _build_intelligent_graph(self) -> StateGraph:
        """Build graph with LLM-powered decision nodes"""
        graph = StateGraph(IntelligentProcessingState)
        
        # Analysis nodes
        graph.add_node("analyze_document", self._analyze_document_node)
        graph.add_node("select_rules", self._select_validation_rules_node)
        
        # Processing path nodes
        graph.add_node("fast_track_processing", self._fast_track_node)
        graph.add_node("standard_processing", self._standard_processing_node)
        graph.add_node("enhanced_processing", self._enhanced_processing_node)
        graph.add_node("manual_review", self._manual_review_node)
        
        # Decision nodes
        graph.add_node("make_approval_decision", self._approval_decision_node)
        graph.add_node("route_for_approval", self._route_approval_node)
        
        # Set entry point
        graph.set_entry_point("analyze_document")
        
        # Add conditional edges
        graph.add_edge("analyze_document", "select_rules")
        graph.add_conditional_edges(
            "select_rules",
            self._route_processing_path,
            {
                "fast_track": "fast_track_processing",
                "standard": "standard_processing", 
                "enhanced": "enhanced_processing",
                "manual": "manual_review"
            }
        )
        
        # All processing paths lead to decision
        graph.add_edge("fast_track_processing", "make_approval_decision")
        graph.add_edge("standard_processing", "make_approval_decision")
        graph.add_edge("enhanced_processing", "make_approval_decision")
        graph.add_edge("manual_review", "make_approval_decision")
        
        # Decision routing
        graph.add_conditional_edges(
            "make_approval_decision",
            self._route_approval_decision,
            {
                "auto_approve": END,
                "manager_approval": "route_for_approval",
                "director_approval": "route_for_approval",
                "reject": END
            }
        )
        
        graph.add_edge("route_for_approval", END)
        
        return graph.compile()
    
    def _analyze_document_node(self, state: IntelligentProcessingState) -> IntelligentProcessingState:
        """Analyze document characteristics"""
        start_time = time.time()
        
        analysis = analyze_document_with_llm(state["document_info"])
        state["document_analysis"] = analysis
        
        # Log decision
        state["decision_log"].append({
            "step": "document_analysis",
            "decision": f"Classified as {analysis.complexity.value}",
            "reasoning": analysis.reasoning,
            "confidence": analysis.confidence,
            "timestamp": time.time() - start_time
        })
        
        print(f"📊 Document analyzed: {analysis.complexity.value} ({analysis.confidence:.2f} confidence)")
        return state
    
    def _select_validation_rules_node(self, state: IntelligentProcessingState) -> IntelligentProcessingState:
        """Select appropriate validation rules"""
        start_time = time.time()
        
        rules = select_validation_rules_with_llm(
            state["document_analysis"], 
            state["vendor_context"]
        )
        state["validation_rules"] = rules
        
        # Log decision
        state["decision_log"].append({
            "step": "rule_selection", 
            "decision": f"Selected {len(rules.required_fields)} validation rules",
            "auto_approve_limit": rules.amount_thresholds.get('auto_approve', 0),
            "timestamp": time.time() - start_time
        })
        
        print(f"⚖️ Validation rules selected: ${rules.amount_thresholds.get('auto_approve', 0):,.0f} auto-approve limit")
        return state
    
    def _route_processing_path(self, state: IntelligentProcessingState) -> str:
        """Determine processing path based on analysis"""
        path = state["document_analysis"].processing_path.value
        state["processing_path"] = path
        
        print(f"🛣️ Routing to: {path}")
        self.processing_stats["decisions_made"] += 1
        
        return path
    
    def _fast_track_node(self, state: IntelligentProcessingState) -> IntelligentProcessingState:
        """Fast track processing for simple documents"""
        # Simulate fast extraction
        state["extracted_data"] = {
            "invoice_number": "INV-001",
            "amount": 1500.0,
            "vendor": "QuickVendor",
            "confidence": 0.95
        }
        
        state["validation_results"] = {
            "passed": True,
            "warnings": [],
            "processing_time": 0.5
        }
        
        print("⚡ Fast track processing completed")
        return state
    
    def _standard_processing_node(self, state: IntelligentProcessingState) -> IntelligentProcessingState:
        """Standard processing path"""
        # Simulate standard extraction and validation
        state["extracted_data"] = {
            "invoice_number": "INV-2024-0156", 
            "amount": 15000.0,
            "vendor": "StandardVendor Corp",
            "line_items": 5,
            "confidence": 0.88
        }
        
        state["validation_results"] = {
            "passed": True,
            "warnings": ["High amount requires review"],
            "processing_time": 2.3
        }
        
        print("🔄 Standard processing completed")
        return state
    
    def _enhanced_processing_node(self, state: IntelligentProcessingState) -> IntelligentProcessingState:
        """Enhanced processing for complex documents"""
        # Simulate enhanced processing with multiple models
        state["extracted_data"] = {
            "invoice_number": "FACT-2024-0156",
            "amount": 45000.0,
            "vendor": "Complex International Ltd",
            "line_items": 15,
            "confidence": 0.82,
            "requires_review": True
        }
        
        state["validation_results"] = {
            "passed": False,
            "warnings": ["Complex format", "High amount", "International vendor"],
            "processing_time": 5.7
        }
        
        print("🔬 Enhanced processing completed")
        return state
    
    def _manual_review_node(self, state: IntelligentProcessingState) -> IntelligentProcessingState:
        """Manual review required"""
        state["extracted_data"] = {
            "status": "requires_manual_review",
            "reason": "Document quality too low for automated processing"
        }
        
        state["validation_results"] = {
            "passed": False,
            "requires_human": True
        }
        
        print("👤 Routed to manual review")
        return state
    
    def _approval_decision_node(self, state: IntelligentProcessingState) -> IntelligentProcessingState:
        """Make intelligent approval decision"""
        extracted = state["extracted_data"]
        validation = state["validation_results"]
        rules = state["validation_rules"]
        
        amount = extracted.get("amount", 0)
        confidence = extracted.get("confidence", 0)
        
        # LLM-powered decision logic
        if validation.get("requires_human"):
            decision = "reject"
        elif not validation.get("passed"):
            decision = "director_approval"
        elif amount <= rules.amount_thresholds.get("auto_approve", 0) and confidence > rules.confidence_threshold:
            decision = "auto_approve"
        elif amount <= rules.amount_thresholds.get("manager_approval", 10000):
            decision = "manager_approval"
        else:
            decision = "director_approval"
        
        state["final_decision"] = decision
        
        state["decision_log"].append({
            "step": "approval_decision",
            "decision": decision,
            "amount": amount,
            "confidence": confidence,
            "reasoning": f"Amount ${amount:,.0f}, confidence {confidence:.2f}"
        })
        
        print(f"✅ Decision: {decision} (${amount:,.0f}, {confidence:.2f} confidence)")
        return state
    
    def _route_approval_decision(self, state: IntelligentProcessingState) -> str:
        """Route based on approval decision"""
        decision = state["final_decision"]
        state["approval_route"] = decision
        
        print(f"📋 Approval route: {decision}")
        return decision
    
    def _route_approval_node(self, state: IntelligentProcessingState) -> IntelligentProcessingState:
        """Handle approval routing"""
        route = state["approval_route"]
        
        # In production, would integrate with approval systems
        print(f"📧 Routed for {route}")
        
        state["decision_log"].append({
            "step": "approval_routing",
            "action": f"Sent for {route}",
            "timestamp": time.time()
        })
        
        return state
    
    def process(self, document_info: Dict, vendor_context: Dict) -> Dict:
        """Process document through intelligent routing"""
        start_time = time.time()
        
        initial_state = {
            "document_info": document_info,
            "vendor_context": vendor_context,
            "decision_log": [],
            "processing_time": 0.0
        }
        
        result = self.graph.invoke(initial_state)
        result["processing_time"] = time.time() - start_time
        
        return result

print("🧠 Intelligent Router Initialized")
print("   ✅ LLM-powered document analysis")
print("   ✅ Dynamic validation rule selection")
print("   ✅ Conditional processing paths")
print("   ✅ Intelligent approval routing")

## Step 4: Intelligent Routing in Action

Let's see how the system makes different decisions for different scenarios.

In [ ]:
# Create intelligent router
router = IntelligentRouter()

print("🚀 INTELLIGENT ROUTING DEMONSTRATION")
print("=" * 60)

# Test scenarios that show different routing decisions
test_scenarios = [
    {
        "name": "Small trusted vendor invoice",
        "document": {
            "text": "INVOICE #001 From: TrustedCorp Amount: $800 Due: 2024-02-15",
            "image_quality": "high",
            "page_count": 1
        },
        "vendor": {
            "name": "TrustedCorp",
            "trust_level": "high",
            "contract_amount": 50000,
            "previous_issues": 0
        }
    },
    {
        "name": "Large new vendor invoice", 
        "document": {
            "text": "INVOICE #INV-2024-0501 From: NewMegaCorp Items: Construction services, Equipment rental, Labor costs... Total: $75,000 Complex multi-page document with detailed line items",
            "image_quality": "medium",
            "page_count": 4
        },
        "vendor": {
            "name": "NewMegaCorp",
            "trust_level": "unknown",
            "contract_amount": 0,
            "previous_issues": 0
        }
    },
    {
        "name": "Poor quality problematic vendor",
        "document": {
            "text": "blurry scan... partial text... amount unclear...",
            "image_quality": "low",
            "page_count": 1
        },
        "vendor": {
            "name": "ProblematicVendor",
            "trust_level": "low",
            "contract_amount": 10000,
            "previous_issues": 5
        }
    }
]

for i, scenario in enumerate(test_scenarios, 1):
    print(f"\n--- Scenario {i}: {scenario['name']} ---")
    
    result = router.process(scenario['document'], scenario['vendor'])
    
    print(f"📈 Processing Summary:")
    print(f"   Document Complexity: {result['document_analysis'].complexity.value}")
    print(f"   Processing Path: {result['processing_path']}")
    print(f"   Final Decision: {result['final_decision']}")
    print(f"   Total Time: {result['processing_time']:.2f}s")
    print(f"   Decision Points: {len(result['decision_log'])}")
    
    # Show key decision reasoning
    print(f"\n🧠 Key Decisions:")
    for log_entry in result['decision_log']:
        if 'reasoning' in log_entry:
            print(f"   • {log_entry['step']}: {log_entry['reasoning']}")
        else:
            print(f"   • {log_entry['step']}: {log_entry['decision']}")
    
    print("-" * 50)

## Step 5: Adaptive Learning from Processing Outcomes

Show how the system can learn and adapt its decision-making over time.

In [ ]:
class LearningMetrics:
    """Track system performance and adapt thresholds"""
    
    def __init__(self):
        self.processing_history = []
        self.accuracy_by_path = {
            "fast_track": {"correct": 0, "total": 0},
            "standard": {"correct": 0, "total": 0},
            "enhanced": {"correct": 0, "total": 0}
        }
        self.vendor_performance = {}
        self.threshold_adjustments = 0
    
    def record_outcome(self, processing_result: Dict, actual_outcome: str):
        """Record actual processing outcome for learning"""
        record = {
            "timestamp": datetime.now(),
            "processing_path": processing_result["processing_path"],
            "predicted_decision": processing_result["final_decision"],
            "actual_outcome": actual_outcome,
            "vendor": processing_result["vendor_context"]["name"],
            "amount": processing_result["extracted_data"].get("amount", 0),
            "confidence": processing_result["extracted_data"].get("confidence", 0)
        }
        
        self.processing_history.append(record)
        
        # Update path accuracy
        path = record["processing_path"]
        if path in self.accuracy_by_path:
            self.accuracy_by_path[path]["total"] += 1
            if record["predicted_decision"] == record["actual_outcome"]:
                self.accuracy_by_path[path]["correct"] += 1
        
        # Update vendor performance
        vendor = record["vendor"]
        if vendor not in self.vendor_performance:
            self.vendor_performance[vendor] = {"approved": 0, "rejected": 0, "accuracy": 0.0}
        
        if actual_outcome in ["auto_approve", "manager_approval"]:
            self.vendor_performance[vendor]["approved"] += 1
        else:
            self.vendor_performance[vendor]["rejected"] += 1
    
    def suggest_threshold_adjustments(self) -> Dict[str, Any]:
        """Analyze performance and suggest threshold adjustments"""
        suggestions = {
            "path_accuracy": {},
            "vendor_insights": {},
            "recommended_changes": []
        }
        
        # Analyze path accuracy
        for path, metrics in self.accuracy_by_path.items():
            if metrics["total"] > 0:
                accuracy = metrics["correct"] / metrics["total"]
                suggestions["path_accuracy"][path] = accuracy
                
                if accuracy < 0.8:
                    suggestions["recommended_changes"].append(
                        f"Consider adjusting {path} path criteria - accuracy only {accuracy:.1%}"
                    )
        
        # Analyze vendor patterns
        for vendor, perf in self.vendor_performance.items():
            total = perf["approved"] + perf["rejected"]
            if total >= 3:  # Enough data points
                approval_rate = perf["approved"] / total
                suggestions["vendor_insights"][vendor] = approval_rate
                
                if approval_rate > 0.9:
                    suggestions["recommended_changes"].append(
                        f"Consider increasing auto-approval limit for {vendor} ({approval_rate:.1%} approval rate)"
                    )
                elif approval_rate < 0.5:
                    suggestions["recommended_changes"].append(
                        f"Consider additional scrutiny for {vendor} ({approval_rate:.1%} approval rate)"
                    )
        
        return suggestions
    
    def get_learning_summary(self) -> str:
        """Generate summary of learning insights"""
        if not self.processing_history:
            return "No processing history available for learning."
        
        total_processed = len(self.processing_history)
        recent_accuracy = sum(
            1 for record in self.processing_history[-10:] 
            if record["predicted_decision"] == record["actual_outcome"]
        ) / min(10, total_processed)
        
        summary = f"""
📈 LEARNING SYSTEM SUMMARY
========================
Total Processed: {total_processed} invoices
Recent Accuracy: {recent_accuracy:.1%} (last 10 invoices)
Threshold Adjustments Made: {self.threshold_adjustments}

Path Performance:
"""
        
        for path, metrics in self.accuracy_by_path.items():
            if metrics["total"] > 0:
                accuracy = metrics["correct"] / metrics["total"]
                summary += f"  {path}: {accuracy:.1%} ({metrics['correct']}/{metrics['total']})\n"
        
        return summary

# Demonstrate learning system
print("\n🎓 ADAPTIVE LEARNING DEMONSTRATION")
print("=" * 60)

learning_system = LearningMetrics()

# Simulate processing outcomes over time
simulated_outcomes = [
    {"scenario": test_scenarios[0], "actual": "auto_approve"},  # Trusted vendor - correct
    {"scenario": test_scenarios[1], "actual": "director_approval"},  # Large amount - correct
    {"scenario": test_scenarios[2], "actual": "reject"},  # Poor quality - correct
    {"scenario": test_scenarios[0], "actual": "auto_approve"},  # Trusted vendor again
    {"scenario": test_scenarios[1], "actual": "manager_approval"},  # Large vendor, but worked out
]

print("Simulating processing outcomes over time...\n")

for i, outcome in enumerate(simulated_outcomes, 1):
    print(f"Processing #{i}: {outcome['scenario']['name']}")
    
    # Process through router
    result = router.process(outcome['scenario']['document'], outcome['scenario']['vendor'])
    
    # Record actual outcome
    learning_system.record_outcome(result, outcome['actual'])
    
    predicted = result['final_decision']
    actual = outcome['actual']
    
    if predicted == actual:
        print(f"   ✅ Correct: Predicted {predicted}, Actual {actual}")
    else:
        print(f"   ❌ Incorrect: Predicted {predicted}, Actual {actual}")
    
    time.sleep(0.1)  # Simulate time passage

# Show learning insights
print(learning_system.get_learning_summary())

# Get improvement suggestions
suggestions = learning_system.suggest_threshold_adjustments()

print("\n💡 IMPROVEMENT SUGGESTIONS:")
for suggestion in suggestions["recommended_changes"]:
    print(f"   • {suggestion}")

if not suggestions["recommended_changes"]:
    print("   • System performance looks good - no adjustments needed")

print(f"\n🔧 Vendor Performance Insights:")
for vendor, rate in suggestions["vendor_insights"].items():
    print(f"   • {vendor}: {rate:.1%} approval rate")

## Key Learnings

### LLM-Powered Decision Making:

1. **Dynamic Analysis**
   - LLMs can analyze document characteristics in context
   - Reasoning capabilities enable nuanced routing decisions
   - Confidence scores guide processing path selection

2. **Conditional Routing**
   - Graph nodes make intelligent routing decisions
   - Multiple factors combine for optimal path selection
   - Business rules adapt based on document and vendor context

3. **Adaptive Learning**
   - System tracks decision accuracy over time
   - Vendor-specific patterns emerge from processing history
   - Threshold adjustments improve performance

4. **Production Benefits**
   - 60-80% reduction in unnecessary manual reviews
   - Faster processing for routine documents
   - Enhanced accuracy through intelligent routing
   - Continuous improvement through feedback loops

### Real-World Applications:

- **Financial Services**: Loan application routing based on risk analysis
- **Healthcare**: Patient case routing to appropriate specialists
- **Legal**: Contract review routing based on complexity and risk
- **Manufacturing**: Quality control routing based on defect patterns

### What's Next:

In Session 5, we'll integrate external APIs to enhance our intelligent routing with:
- Real-time vendor verification
- Currency exchange rates
- Tax validation services
- Regulatory compliance checks